# Explaining our Data Types
What are data types? In imprecise terms, they are wrappers around attributes or values that one can define computational behavior on. In the Julia programming language, one defines data types via a `struct`. 

Our most important data type is `PauliSum`, which is a wrapper for a sum of Pauli strings. What is a Pauli string? Mathematically they are a tensor product of single-qubit Pauli matrices. For example, $I \otimes X \otimes Z$ is a 3-qubit Pauli string.

In [1]:
using PauliPropagation

Let us start by defining an empty `PauliSum` on 3 qubits.

In [18]:
nqubits = 3

psum = PauliSum(nqubits)

PauliSum(nqubits: 3, (no Pauli strings))

If we inspect the Pauli sum `psum`, it carries two attributes: `nqubits` and `terms`.

In [19]:
psum.nqubits

3

In [20]:
psum.terms

Dict{UInt8, Float64}()

`terms` are the collection of Pauli strings and their respective coefficients. We store them as a dictionary, which is currently empty. It says the type of that dictionary is `Dict{UInt8, Float64`, and we will see what that means.

Let us now add terms to the Pauli sum. We simply do this by calling the `add!()` function on `psum` with some extra information about the Pauli string that we want to add.

In [21]:
add!(psum, :X, 2)  # this adds 1.0 * IXI
add!(psum, [:Y, :Z], [1, 3], 0.5)  # this adds 0.5 * YIZ

psum  # the display order usually does not match the order in which you added the terms, but that is fine.

PauliSum(nqubits: 3, 2 Pauli terms:
 1.0 * IXI
 0.5 * YIZ
)

Note that in the display qubits are indexed from left to right.

We also have a `PauliString` data type, that can be nice to use at a high level. It can be created in the same ways that you can add to a `PauliSum` via `add!()`.

In [22]:
pstr = PauliString(nqubits, [:X, :Y], [2, 3])

PauliString(nqubits: 3, 1.0 * IXY)

`PauliString` has very similar attributes, `nqubits` and `term`. The latter is our efficient low-level implementation.

Internally we will always convert to `PauliSum`, but `PauliString` may be convenient for you. 

In [23]:
# this works
psum - pstr

# this also works, and modifies psum in-place
add!(psum, pstr)

PauliSum(nqubits: 3, 3 Pauli terms:
 1.0 * IXI
 0.5 * YIZ
 1.0 * IXY
)

Let us now dig a bit deeper and look at the terms:

In [24]:
psum.terms

Dict{UInt8, Float64} with 3 entries:
  0x04 => 1.0
  0x32 => 0.5
  0x24 => 1.0

What are `0x04`, `0x32`, `0x24`? They are our low-level implementation of Pauli strings as unsigned integers, here as 8-bit unsigned integers `UInt8`. We encode the Pauli strings in the bits of an integer.

In [26]:
println(0x4)
println(0x32)
println(0x24)

4
50
36


With the `Bits` package we can easily display the bits of the unsigned integer. Note that there qubits are indexed from right to left, but you do not need to remember this if you use our functions.

In [27]:
using Bits
bits(0x32)

<00110010>

The above shows that the first Pauli (on the right) is `10` (the number 2), which is `Y`. The second is `00` (the number 0), which is `I`. The third is `11` (the number 3), which is `Z`. All bits beyond `2 * nqubits` will be zero. Under the hood, we try to use the smallest integer type that can carry the Pauli strings. 

This can also be checked in the following way:

In [28]:
# use `getpauli(pstr, qind) to get the Paulis as 0, 1, 2, or 3 on the site `qind`
getpauli(0x32, 1) == 2, getpauli(0x32, 2) == 0, getpauli(0x32, 3) == 3

(true, true, true)

`getpauli()` and `setpauli()` are very useful functions for when you want to define your own gates. As a quick example:

In [29]:
pstr = 0x32
@show bits(pstr)

# set Pauli at qind 3 to Y
qind = 3
pauli = 2 # Y, 0x02 also works
pstr_after = setpauli(pstr, pauli, qind)

@show bits(pstr_after)

pstr_after

bits(pstr) = <00110010>
bits(pstr_after) = <00100010>


0x22